In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys 
import os
from PIL import Image

In [10]:
!pip install pillow

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
list_ds = tf.data.Dataset.list_files(str('../bean_data/*/*'))

In [12]:
for x in list_ds.take(1):
    print(x)

tf.Tensor(b'../bean_data/angular leaf spot of beans/Image_243.jpg', shape=(), dtype=string)


In [13]:
def parse_image(filename):
  parts = tf.strings.split(filename, os.sep)
  label = parts[-2]

  image = tf.io.read_file(filename)
  try:
      image = tf.image.decode_jpeg(image, channels=3)
  except Exception as e:
    print(e)
    print(filename)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize_with_pad(image, 224, 224)
  return image, label

In [14]:
for i, file_path in enumerate(list_ds):
    if i % 100 == 0:
        print(f"processing file {i} filepath {file_path}")
    image, label = parse_image(file_path)
    


processing file 0 filepath b'../bean_data/Phaeoisariopsis griseola beans/Image_394.png'
processing file 100 filepath b'../bean_data/angular leaf spot of beans/Image_120.jpg'
processing file 200 filepath b'../bean_data/Phaeoisariopsis griseola beans/Image_11.jpg'
processing file 300 filepath b'../bean_data/Phaseolus vulgaris leaves/Image_421.jpg'
processing file 400 filepath b'../bean_data/bean leaves/Image_6.JPG'
processing file 500 filepath b'../bean_data/Phaseolus vulgaris leaves/Image_224.jpg'
processing file 600 filepath b'../bean_data/angular leaf spot of beans/Image_347.jpg'
processing file 700 filepath b'../bean_data/Phaseolus vulgaris leaves/Image_389.jpg'
processing file 800 filepath b'../bean_data/Phaseolus vulgaris leaves/Image_434.jpg'
processing file 900 filepath b'../bean_data/bean leaves/Image_25.jpg'
processing file 1000 filepath b'../bean_data/Phaseolus vulgaris leaves/Image_479.jpg'
processing file 1100 filepath b'../bean_data/bean leaves/Image_185.jpg'
processing fil